<a href="https://colab.research.google.com/github/BernardoRMendes/Projeto/blob/main/ValuationFinancials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Nesse estudo iremos calcular o valor justo de ações do setor financeiro usando um modelo de Regressão Múltipla utilizando os tradicionais indicadores P/L (Preço sobre Lucro), P/VP (Preço sobre Valor Patrimonial) e PSR (Preço sobre Receira). Os dados foram coletados do site statusinvest e salvos como arquivo csv.

In [2]:
url="https://raw.githubusercontent.com/BernardoRMendes/Projeto/main/Valuation.csv"

In [3]:
df=pd.read_csv(url,sep=';',index_col=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, BBDC4 to WIZS3
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRECO   16 non-null     float64
 1   P/L     16 non-null     float64
 2   P/VP    16 non-null     float64
 3   PSR     16 non-null     float64
dtypes: float64(4)
memory usage: 640.0+ bytes


In [5]:
df.head()

,PRECO,P/L,P/VP,PSR
TICKER,,,,
BBDC4,25.01,15.24,1.61,2.08
ITUB4,29.27,15.44,2.11,1.57
BBAS3,33.69,6.04,0.86,0.77
B3SA3,58.55,31.83,4.83,15.21
IRBR3,6.92,-13.50,1.80,1.08


Como as ações possuem preços diferentes, precisamos normalizar seus valores para que a regressão não tenha problemas de heterocedasticidade. Além disso, adicionaremos a constante do modelo manualmente.

In [6]:
df["PRECO_Normalizado"]=df["PRECO"]/df["PRECO"]
df["Earnings Yield"]=1/df["P/L"]
df["VP/P"]=1/df["P/VP"]
df["RSP"]=1/df["PSR"]
df["Const"]=1/df["PRECO"]

In [7]:
df

,PRECO,P/L,P/VP,PSR,PRECO_Normalizado,Earnings Yield,VP/P,RSP,Const
TICKER,,,,,,,,,
BBDC4,25.01,15.24,1.61,2.08,1.0,0.065617,0.621118,0.480769,0.039984
ITUB4,29.27,15.44,2.11,1.57,1.0,0.064767,0.473934,0.636943,0.034165
BBAS3,33.69,6.04,0.86,0.77,1.0,0.165563,1.162791,1.298701,0.029682
B3SA3,58.55,31.83,4.83,15.21,1.0,0.031417,0.207039,0.065746,0.017079
IRBR3,6.92,-13.50,1.80,1.08,1.0,-0.074074,0.555556,0.925926,0.144509
BPAC11,92.88,29.44,3.23,6.45,1.0,0.033967,0.309598,0.155039,0.010767
SANB11,40.60,10.04,1.47,2.32,1.0,0.099602,0.680272,0.431034,0.024631
CIEL3,3.61,22.20,1.07,0.92,1.0,0.045045,0.934579,1.086957,0.277008
BBDC3,21.92,13.36,1.41,1.82,1.0,0.074850,0.709220,0.549451,0.045620


In [8]:
x=df[["Earnings Yield","VP/P","RSP","Const"]]
y=df["PRECO_Normalizado"]
modelo=sm.OLS(y, x)
resultado=modelo.fit()
print(resultado.summary())

                                 OLS Regression Results                                
Dep. Variable:      PRECO_Normalizado   R-squared (uncentered):                   0.835
Model:                            OLS   Adj. R-squared (uncentered):              0.780
Method:                 Least Squares   F-statistic:                              15.15
Date:                Mon, 25 Jan 2021   Prob (F-statistic):                    0.000123
Time:                        01:03:39   Log-Likelihood:                         -8.3038
No. Observations:                  16   AIC:                                      24.61
Df Residuals:                      12   BIC:                                      27.70
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


Como o modelo possui poucas observações, já é esperado que os coeficientes tenham pouco poder estatístico. Como ponto positivo temos um Adj. R-Squared de 0.78, indicando que o modelo explica 78% da variável dependente.

In [9]:
resultado.params

Earnings Yield    2.732533
VP/P              0.394263
RSP               0.190362
Const             1.944698
dtype: float64

In [10]:
df["Valor"]=resultado.params[0]*df["Earnings Yield"]+resultado.params[1]*df["VP/P"]+resultado.params[2]*df["RSP"]+resultado.params[3]*df["Const"]

In [11]:
df

,PRECO,P/L,P/VP,PSR,PRECO_Normalizado,Earnings Yield,VP/P,RSP,Const,Valor
TICKER,,,,,,,,,,
BBDC4,25.01,15.24,1.61,2.08,1.0,0.065617,0.621118,0.480769,0.039984,0.593461
ITUB4,29.27,15.44,2.11,1.57,1.0,0.064767,0.473934,0.636943,0.034165,0.551522
BBAS3,33.69,6.04,0.86,0.77,1.0,0.165563,1.162791,1.298701,0.029682,1.215798
B3SA3,58.55,31.83,4.83,15.21,1.0,0.031417,0.207039,0.065746,0.017079,0.213206
IRBR3,6.92,-13.50,1.80,1.08,1.0,-0.074074,0.555556,0.925926,0.144509,0.473912
BPAC11,92.88,29.44,3.23,6.45,1.0,0.033967,0.309598,0.155039,0.010767,0.265331
SANB11,40.60,10.04,1.47,2.32,1.0,0.099602,0.680272,0.431034,0.024631,0.670322
CIEL3,3.61,22.20,1.07,0.92,1.0,0.045045,0.934579,1.086957,0.277008,1.237170
BBDC3,21.92,13.36,1.41,1.82,1.0,0.074850,0.709220,0.549451,0.045620,0.677463


In [12]:
df["Preço Justo"]=df["PRECO"]*df["Valor"]

In [13]:
df

,PRECO,P/L,P/VP,PSR,PRECO_Normalizado,Earnings Yield,VP/P,RSP,Const,Valor,Preço Justo
TICKER,,,,,,,,,,,
BBDC4,25.01,15.24,1.61,2.08,1.0,0.065617,0.621118,0.480769,0.039984,0.593461,14.842457
ITUB4,29.27,15.44,2.11,1.57,1.0,0.064767,0.473934,0.636943,0.034165,0.551522,16.143043
BBAS3,33.69,6.04,0.86,0.77,1.0,0.165563,1.162791,1.298701,0.029682,1.215798,40.960245
B3SA3,58.55,31.83,4.83,15.21,1.0,0.031417,0.207039,0.065746,0.017079,0.213206,12.483183
IRBR3,6.92,-13.50,1.80,1.08,1.0,-0.074074,0.555556,0.925926,0.144509,0.473912,3.279472
BPAC11,92.88,29.44,3.23,6.45,1.0,0.033967,0.309598,0.155039,0.010767,0.265331,24.643948
SANB11,40.60,10.04,1.47,2.32,1.0,0.099602,0.680272,0.431034,0.024631,0.670322,27.215084
CIEL3,3.61,22.20,1.07,0.92,1.0,0.045045,0.934579,1.086957,0.277008,1.237170,4.466183
BBDC3,21.92,13.36,1.41,1.82,1.0,0.074850,0.709220,0.549451,0.045620,0.677463,14.849978


In [14]:
df["Upside/Downside"]=df["Preço Justo"]/df["PRECO"]-1

In [15]:
(df["Upside/Downside"]*100).round(2).astype(str) + '%'

TICKER
BBDC4     -40.65%
ITUB4     -44.85%
BBAS3      21.58%
B3SA3     -78.68%
IRBR3     -52.61%
BPAC11    -73.47%
SANB11    -32.97%
CIEL3      23.72%
BBDC3     -32.25%
SULA11     -2.26%
PSSA3     -24.02%
BRSR6      40.98%
BPAN4     -27.62%
ABCB4      21.68%
BMGB4      41.91%
WIZS3      -4.99%
Name: Upside/Downside, dtype: object

Obviamente, não é condizente com a realidade estimar preços justos de -78,68% para B3SA3 e -73,47% para BPAC11. No entanto, desconsiderando a "intensidade" do valor estimado, é plausível supor que essas instituições financeiras estejam com valuation em patamar sobrevalorizado, quando comparados os múltiplos.
Além disso, tanto o Banco do Brasil (BBAS3) quanto o Banrisul (BRSR6) estão com o preço atual abaixo do valor justo, indicando que o mercado exige um prêmio pelo controle estatal.